In [1]:
#ChatGPT
# chf_differential_privacy.py

# Step 0: Imports
import pandas as pd
import numpy as np
from diffprivlib.mechanisms import Laplace

# Step 1: Load dataset
csv_path = "/Users/felipecastanogonzalez/Downloads/ChfSynthData-13_09_2024.csv"
df = pd.read_csv(csv_path)

# Step 2: Filter and clean data
df = df.drop_duplicates()
df = df.dropna(subset=["FirstName", "LastName", "DobDay", "DobMonth", "DobYear"])
df["DobYear"] = df["DobYear"].astype(int)

# Step 3: Keep only required columns
df = df[["FirstName", "LastName", "DobDay", "DobMonth", "DobYear"]]

# Step 4: Create 'Decade' column
df["Decade"] = (df["DobYear"] // 10) * 10

# Step 5: Count records per decade (true counts)
true_counts = df["Decade"].value_counts().sort_index()

# Step 6: Apply Laplace mechanism for Differential Privacy
epsilon = 0.1
sensitivity = 1

dp_counts = []
for count in true_counts:
    lap = Laplace(epsilon=epsilon, sensitivity=sensitivity)
    lap.randomise(count)
    noisy_count = max(0, int(round(lap.randomise(count))))
    dp_counts.append(noisy_count)

# Step 7: Create comparison DataFrame
comparison_df = pd.DataFrame({
    "Decade": true_counts.index,
    "True Count": true_counts.values,
    "DP Count (ε=0.1)": dp_counts
})

# Step 8: Print comparison table
print(comparison_df.to_string(index=False))

 Decade  True Count  DP Count (ε=0.1)
   1900        1650              1649
   1910        1691              1703
   1920        1500              1506
   1930        1572              1572
   1940        1692              1709
   1950        1625              1630
   1960        1433              1413
   1970        1565              1565
   1980        1626              1634
   1990        1633              1644
   2000           2                 0
   2010           2                 6
